In [1]:
from funcx.sdk.client import FuncXClient
import time


fxc = FuncXClient()
fxc.throttling_enabled = False
print(fxc.base_url, fxc.throttling_enabled)

Please paste the following URL in a browser:
https://auth.globus.org/v2/oauth2/authorize?client_id=4cf29807-cf21-49ec-9443-ff9a3fb9f81c&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2Ffacd7ccc-c5f4-42aa-916b-a0e270e2c2a9%2Fall&state=_default&response_type=code&code_challenge=nCbLmcqPdF5r8VcgdozLKNgIJ-cYE1rNpH6GWGXK16M&code_challenge_method=S256&access_type=offline&prefill_named_grant=FuncX+SDK+Login
Please Paste your Auth Code Below: 
320UyOyNEoRmKGmRdS4EWQEvTW0IvB
https://funcx.org/api/v1 False


# Functions to work with endpoints

In [2]:
endpoints = ['709118de-1103-463f-8425-281eb93b55ff',   #Theta covid19
             '67e95158-8bda-4b1f-a0ef-31a1626eba00',   #Ryan ep
            ]

In [35]:
def get_endpoints_status(endpoints):
    endpoint_status = {}
    idle_workers = {}
    for ep in endpoints:
        endpoint_status[ep] = fxc.get_endpoint_status(ep)[0]
        idle_workers[ep] = endpoint_status[ep]['idle_workers']
    return endpoint_status, idle_workers

# fx functions

In [34]:
def warm_endpoint():
    """A funcX function to warm the endpoint 
    and create workers"""
    pass

In [38]:
def funcx_runner(index, filename, batchsize, index_start, index_end, workdir=None, out_file=None):
    """funcX function to perform work"""
    from candle_apps.candle_node_local import funcx_node_local
    return funcx_node_local(filename, index, batchsize, index_start, index_end,
                            workdir=workdir, out_file=out_file)

In [83]:
func_uuid2 = fxc.register_function(funcx_runner,
                                   description="A funcx function for covid19")

print(func_uuid2)

c739e938-4dc0-488c-ac5d-d81fb808f636


# Gerate workloads

In [50]:
def generate_batch(filename, start=0, batchsize=10, max_batches=10):
    counter = 0
    if max_batches == 0:
        max_batches = 999999999

    x = 'Hello'
    batch_index = []
    with open(filename) as current:
        batch_index.append(current.tell())
        counter += 1

        while x and counter < max_batches:
            counter += 1
            for i in range(batchsize):
                x = current.readline()

            batch_index.append(current.tell())
        return batch_index

In [51]:
func_uuid1 = fxc.register_function(generate_batch,
                                   description="A funcx function for generating batch")

print(func_uuid1)

0cf5044f-0b5e-4bcd-a102-2eb48974381d


In [65]:
res1 = fxc.run(filename='/home/rchard/src/covid19/ScreenPilot/ena+db.can',
               endpoint_id='67e95158-8bda-4b1f-a0ef-31a1626eba00',
               function_id=func_uuid1)

print(res1)

396ac761-9def-44e0-a5c7-f4a7432d48af


In [66]:
batch_index = fxc.get_result(res1)
print(batch_index)

[0, 2321, 3153, 3480, 4006, 4429, 4950, 5357, 5994, 6402]


# Distribute work

In [69]:
stats, idle = get_endpoints_status(endpoints)

In [70]:
idle

{'709118de-1103-463f-8425-281eb93b55ff': 1,
 '67e95158-8bda-4b1f-a0ef-31a1626eba00': 2}

Define configs for the two eps

In [56]:
configs = {'709118de-1103-463f-8425-281eb93b55ff': {'filename': '/home/zzli/candle/ScreenPilot/ena+db.can',
                                                    'workdir': '/tmp/zzli/',
                                                    'out_path': '/home/zzli/candle/ScreenPilot/'},
           '67e95158-8bda-4b1f-a0ef-31a1626eba00': {'filename': '/home/rchard/src/covid19/ScreenPilot/ena+db.can',
                                                    'workdir': '/tmp/rchard/',
                                                    'out_path': '/home/rchard/src/covid19/ScreenPilot/'}
          }

Submit workloads

In [93]:
def submit_work(endpoint_uuid, idle_workers, idx):
    exec_config = configs[ep]
    res = fxc.run(filename=exec_config['filename'],
              index=idx,
              batchsize=10,
              index_start=0,
              index_end=1000,
              workdir=exec_config['workdir'],
              out_file=f'{exec_config["out_path"]}data-{idx}',
              endpoint_id=endpoint_uuid,
              function_id=func_uuid2)
    return res

In [94]:
task_ids = []
for ep, idle_workers in idle.items():
    for x in range(idle_workers):
        idx = batch_index.pop()
        print(f'submitting {idx} to {ep}')
        task_ids.append(submit_work(ep, idle, idx))

submitting 3480 to 709118de-1103-463f-8425-281eb93b55ff
submitting 3153 to 67e95158-8bda-4b1f-a0ef-31a1626eba00
submitting 2321 to 67e95158-8bda-4b1f-a0ef-31a1626eba00


In [95]:
print(task_ids)

['124b10ab-c2a1-4445-af62-407da76e57fb', '7b755b9e-981f-45ee-9dc6-f9f28646417f', 'b3630c49-91cb-4972-b4fc-421b8e1a7cdd']


In [96]:
x = fxc.get_batch_status(task_ids)
print(x)
try:
    print(x[res[0]]['exception'].reraise())
except:
    pass

{'124b10ab-c2a1-4445-af62-407da76e57fb': {'pending': 'False', 'result': '/home/zzli/candle/ScreenPilot/data-3480'}, '7b755b9e-981f-45ee-9dc6-f9f28646417f': {'pending': 'False', 'result': '/home/rchard/src/covid19/ScreenPilot/data-3153'}, 'b3630c49-91cb-4972-b4fc-421b8e1a7cdd': {'pending': 'False', 'result': '/home/rchard/src/covid19/ScreenPilot/data-2321'}}
